# Variables

1. NearestMonth - Front/Second/etc. month at each respective date - starts from 1
2. backAdjustDays - how many days back we need to adjust       # 0 means expired on the day of expiry

In [1]:
import os
import pandas as pd
import numpy as np
from os import walk
import collections

In [2]:
# Path of all files in folder and subfolder - CONTINUE Folder
List_of_files_continue = []
folder_continue = "C:/source/2x4_data/app/data/Barchart/DailyDataDumpCME&NYMEX/dailyContinue/"
folder_names_continue = [os.path.join(folder_continue, name) for name in os.listdir(folder_continue) if os.path.isdir(os.path.join(folder_continue, name))]
folder_names_continue.append(folder_continue)
for fold in folder_names_continue:
    filenames = next(walk(fold), (None, None, []))[2]  # [] if no file
    for file in filenames:
        List_of_files_continue.append(str(fold+"/"+file).replace('//','/'))
List_of_files_continue = [val for val in List_of_files_continue if not val.endswith(".ipynb")] # remove all files end in .ipynb
List_of_files_continue = [val for val in List_of_files_continue if not val.endswith(".pkl")] # remove all files end in .pkl
List_of_files_continue = [val for val in List_of_files_continue if not val.endswith("expiry_df_other_symbols_Continue.csv")]
List_of_files_continue = [val for val in List_of_files_continue if not val.endswith("expiry_df_Continue.csv")]

len(List_of_files_continue)

621

In [3]:
# Path of all files in folder and subfolder - NEAREST Folder
List_of_files_nearest = []
folder_nearest = "C:/source/2x4_data/app/data/Barchart/DailyDataDumpCME&NYMEX/"
folder_names_nearest = [os.path.join(folder_nearest, name) for name in os.listdir(folder_nearest) if os.path.isdir(os.path.join(folder_nearest, name))]
folder_names_nearest.append(folder_nearest)
folder_names_nearest.remove("C:/source/2x4_data/app/data/Barchart/DailyDataDumpCME&NYMEX/dailyContinue")
folder_names_nearest.remove("C:/source/2x4_data/app/data/Barchart/DailyDataDumpCME&NYMEX/Archived CSV Files")
for fold in folder_names_nearest:
    filenames = next(walk(fold), (None, None, []))[2]  # [] if no file
    for file in filenames:
        List_of_files_nearest.append(str(fold+"/"+file).replace('//','/'))
List_of_files_nearest = [val for val in List_of_files_nearest if not val.endswith(".ipynb")] # remove all files end in .ipynb
List_of_files_nearest = [val for val in List_of_files_nearest if not val.endswith(".pkl")] # remove all files end in .pkl
List_of_files_nearest = [val for val in List_of_files_nearest if not val.endswith("combined_dataframe.csv")]
List_of_files_nearest = [val for val in List_of_files_nearest if not val.endswith("expiry_df.csv")]
List_of_files_nearest = [val for val in List_of_files_nearest if not val.endswith("expiry_df_other_symbols.csv")]

len(List_of_files_nearest)

618

In [4]:
# Matching Nearest & Continue FILES - just remove "_Continue" at the end but before ".csv"
# For each file in Nearest, change the string of the path to make it similar to one in Continue and the check if its exits in Continue

file_paths_dict = {}
for file_nearest in List_of_files_nearest:
    continue_string = file_nearest[file_nearest.rindex('/')+1:]
    continue_string = str(continue_string.replace(".csv","")+"_Continue.csv")
    idx_continue = [i for i, s in enumerate(List_of_files_continue) if continue_string in s]
    file_paths_dict[file_nearest] = List_of_files_continue[idx_continue[0]]

len(file_paths_dict.keys())    
        

618

In [5]:
# file_paths_dict['C:/source/2x4_data/app/data/Barchart/DailyDataDumpCME&NYMEX/Archived CSV Files/Australian Dollar(P)_daily_price.csv']

In [6]:
# Get Expiry dataframe from csv
expiry_nearest = pd.read_csv("C:/source/2x4_data/app/data/Barchart/DailyDataDumpCME&NYMEX/expiry_df.csv", index_col=0)
expiry_nearest.head(2)   

,symbol,exchange,contract,firstNoticeDate,expirationDate,lastTradingDay,Unnamed: 7
MGM22,MGM22,GBLX,_Jun 22,6/13/2022,6/13/2022,Second business day preceding third Wednesday ...,NaN
MGH22,MGH22,GBLX,_Mar 22,3/14/2022,3/14/2022,Second business day preceding third Wednesday ...,NaN


In [7]:
# # file_paths_dict
# import difflib
# Main_folder_string = 'C:/source/2x4_data/app/data/Barchart/DailyDataDumpCME&NYMEX/'


# for file in list(file_paths_dict.keys())[:10]:
#     cmdty_name = file[file.rindex('/')+1:].replace('_daily_price.csv','')
#     res=''.join(cmdty_name.split(Main_folder_string))
#     print(res)

In [8]:
# Collect all contracts for all commodities in a dictionary and a list of dates
all_dates = []
cmdty_ctrct_dict = {}
cmdty_no_data = []
temp_list = []
cmdty_no_expiry = []
cmdty_ctrct_expiry_dict = collections.defaultdict(dict) # If the amount of nesting you need is fixed, use collections.defaultdict.
file_path_cmdty_name = {}

for file in file_paths_dict.keys():
    cmdty_name = file[file.rindex('/')+1:].replace('_daily_price.csv','')
    
    try:
        list_nearest_ctrct = pd.read_csv(file)['symbol'].unique().tolist()
        list_nearest_dates = pd.read_csv(file)['tradingDay'].unique().tolist()

    except:
        cmdty_no_data.append(cmdty_name)
        temp_list.append(file_paths_dict[file])

    try:
        list_continue_ctrct = pd.read_csv(file_paths_dict[file],index_col=0)['symbol'].unique().tolist()
        list_continue_dates = pd.read_csv(file_paths_dict[file],index_col=0)['tradingDay'].unique().tolist()

    except:
        cmdty_no_data.append(cmdty_name)
        temp_list.append(file_paths_dict[file])

    cmdty_ctrct_dict[cmdty_name] = set(list_nearest_ctrct + list_continue_ctrct)
    file_path_cmdty_name[file] = cmdty_name

    all_dates = all_dates + list_nearest_dates + list_continue_dates

    for ctrcts in cmdty_ctrct_dict[cmdty_name]:
        try:
            cmdty_ctrct_expiry_dict[cmdty_name][ctrcts] = expiry_nearest.loc[ctrcts,'expirationDate'], expiry_nearest.loc[ctrcts,'exchange'], expiry_nearest.loc[ctrcts,'contract'].replace('_','')
        except:
            cmdty_no_expiry.append(ctrcts)

        # To do dict of dicts
	
temp_list =  set(temp_list)
all_dates = set(all_dates)
cmdty_no_data = set(cmdty_no_data)
all_dates = pd.Series(pd.DatetimeIndex(all_dates))
all_dates =  all_dates.sort_values(ascending=False).dt.strftime('%Y-%m-%d').tolist()  


In [9]:
cmdty_name

'WTS (Argus) vs. WTI Spread Trade Month Swap'

In [10]:
# file_path_cmdty_name['C:/source/2x4_data/app/data/Barchart/DailyDataDumpCME&NYMEX/Archived CSV Files/Cheese Block_daily_price.csv']

In [11]:

len(cmdty_ctrct_dict.keys()), len(list(cmdty_ctrct_expiry_dict.keys())), len(cmdty_no_expiry)

(618, 561, 6141)

In [12]:
cmdty_ctrct_expiry_dict['AUD_USD Micro']['MGZ20'][2]

'Dec 20'

In [13]:
len(cmdty_no_data), len(file_paths_dict.keys()), len(cmdty_ctrct_dict.keys())

(110, 618, 618)

In [14]:
list(file_paths_dict.keys())[10]

'C:/source/2x4_data/app/data/Barchart/DailyDataDumpCME&NYMEX/CHF/CHF_USD Micro_daily_price.csv'

Pass a key index such as list(file_paths_dict.keys())[0]

# In the list above look for duplicates and keep the file which is larger.

In [15]:
LS_path = 'C:/source/2x4_data/app/data/Barchart/DailyDataDumpCME&NYMEX/Lumber_daily_price.csv'
file = LS_path

combined = pd.concat([pd.read_csv(file,index_col=0), pd.read_csv(file_paths_dict[file],index_col=0)])
combined.drop_duplicates(inplace=True)
len(combined)


157047

In [16]:
# file_paths_dict['C:/source/2x4_data/app/data/Barchart/DailyDataDumpCME&NYMEX/Archived CSV Files/3-Month BSBY_daily_price.csv']

Need a way to connect 

In [17]:
# cmdty_ctrct_expiry_dict

In [18]:
one_commodity_expiry = pd.DataFrame(cmdty_ctrct_expiry_dict['Lumber']).T
one_commodity_expiry.columns = ['expiry','exchange','contract']
one_commodity_expiry.expiry = pd.to_datetime(one_commodity_expiry.expiry)
one_commodity_expiry.sort_values(by=['expiry'], ascending= False, inplace=True)
len(one_commodity_expiry)


321

In [19]:
combined_pivot = {}
new_combined_pivot = {}
for ohlcv_opt in ['open','high','low','close','volume','openInterest']:
    combined_pivot[ohlcv_opt] = pd.pivot_table(combined, values=ohlcv_opt, index='tradingDay', columns='symbol').sort_index(ascending=False)
    combined_pivot[ohlcv_opt].reindex(one_commodity_expiry.index, axis=1)
    
    # same as commodity_pivot but added an extra row at the top for the expiry date
    new_combined_pivot[ohlcv_opt] = pd.concat([pd.DataFrame(index = one_commodity_expiry.index, data=list(one_commodity_expiry['expiry']),columns=['Expiry']).T,combined_pivot[ohlcv_opt]], axis=0)


In [20]:
# Takes a lot of Time - ~ 2minutes for 1 commodity because OF nsmallest(NearestMonth), sort_values is faster ~48 seconds
# Get Front month next at each date
NearestMonth = 1  # starts from month 1 
NearestContract_df = pd.DataFrame()

# It doesn't matter if we use 'close' instead of OHLCV below since we just need to find the nearest contract
for dt in new_combined_pivot['close'].index[1:]:
    comp = pd.to_datetime(new_combined_pivot['close'].loc['Expiry'])[dt <= pd.to_datetime(new_combined_pivot['close'].loc['Expiry'])] - pd.to_datetime(dt)
    # active = comp.nsmallest(NearestMonth).index[NearestMonth-1]
    active = comp.sort_values().head(NearestMonth).index[NearestMonth-1]
    temp3 = pd.DataFrame(data=[[dt,active]], columns=['dt','active'])
    NearestContract_df = pd.concat([NearestContract_df,temp3])

In [21]:
# Taking 40 seconds
for ohlcv_opt in ['open','high','low','close','volume','openInterest']:
    NearestContract_df[ohlcv_opt] = np.nan
    NearestContract_df.reset_index(inplace = True)
    NearestContract_df.drop('index', axis=1,inplace = True)
    for i in range(len(NearestContract_df)):
        NearestContract_df.loc[i, ohlcv_opt] = new_combined_pivot[ohlcv_opt].loc[NearestContract_df.iloc[i][0],NearestContract_df.iloc[i][1]]


NearestContract_df

,dt,active,open,high,low,close,volume,openInterest
0,2022-05-10,LSK22,1008.00,1025.00,1008.0,1022.0,53.0,0.0
1,2022-05-09,LSK22,1000.10,1015.00,975.0,1009.0,89.0,163.0
2,2022-05-06,LSK22,990.00,1005.00,985.0,1000.1,147.0,237.0
3,2022-05-05,LSK22,1010.00,1033.40,993.8,1020.0,131.0,340.0
4,2022-05-04,LSK22,1021.60,1025.50,980.0,1010.0,144.0,443.0
...,...,...,...,...,...,...,...,...
13255,1969-10-07,LSH70,78.50,79.00,78.0,79.0,0.0,0.0
13256,1969-10-06,LSH70,80.00,80.50,80.0,80.0,0.0,0.0
13257,1969-10-03,LSH70,80.00,80.00,80.0,80.0,0.0,0.0
13258,1969-10-02,LSH70,80.25,80.50,79.5,80.5,0.0,0.0


In [22]:
# Takes a lot of Time - ~ 2minutes for 1 commodity because OF nsmallest(NearestMonth), sort_values is faster ~48 seconds
# Get Front month next at each date
NearestMonth = 2  # starts from month 1 
Second_NearestContract_df = pd.DataFrame()

# It doesn't matter if we use 'close' instead of OHLCV below since we just need to find the nearest contract
for dt in new_combined_pivot['close'].index[1:]:
    comp = pd.to_datetime(new_combined_pivot['close'].loc['Expiry'])[dt <= pd.to_datetime(new_combined_pivot['close'].loc['Expiry'])] - pd.to_datetime(dt)
    # active = comp.nsmallest(NearestMonth).index[NearestMonth-1]
    active = comp.sort_values().head(NearestMonth).index[NearestMonth-1]
    temp3 = pd.DataFrame(data=[[dt,active]], columns=['dt','active'])
    Second_NearestContract_df = pd.concat([Second_NearestContract_df,temp3])

In [23]:
# Taking 40 seconds
for ohlcv_opt in ['open','high','low','close','volume','openInterest']:
    Second_NearestContract_df[ohlcv_opt] = np.nan
    Second_NearestContract_df.reset_index(inplace = True)
    Second_NearestContract_df.drop('index', axis=1,inplace = True)
    for i in range(len(Second_NearestContract_df)):
        Second_NearestContract_df.loc[i, ohlcv_opt] = new_combined_pivot[ohlcv_opt].loc[Second_NearestContract_df.iloc[i][0],Second_NearestContract_df.iloc[i][1]]


In [24]:
NearestContract_df

,dt,active,open,high,low,close,volume,openInterest
0,2022-05-10,LSK22,1008.00,1025.00,1008.0,1022.0,53.0,0.0
1,2022-05-09,LSK22,1000.10,1015.00,975.0,1009.0,89.0,163.0
2,2022-05-06,LSK22,990.00,1005.00,985.0,1000.1,147.0,237.0
3,2022-05-05,LSK22,1010.00,1033.40,993.8,1020.0,131.0,340.0
4,2022-05-04,LSK22,1021.60,1025.50,980.0,1010.0,144.0,443.0
...,...,...,...,...,...,...,...,...
13255,1969-10-07,LSH70,78.50,79.00,78.0,79.0,0.0,0.0
13256,1969-10-06,LSH70,80.00,80.50,80.0,80.0,0.0,0.0
13257,1969-10-03,LSH70,80.00,80.00,80.0,80.0,0.0,0.0
13258,1969-10-02,LSH70,80.25,80.50,79.5,80.5,0.0,0.0


In [25]:
Second_NearestContract_df

,dt,active,open,high,low,close,volume,openInterest
0,2022-05-10,LSN22,807.70,829.50,799.1,801.10,166.0,0.0
1,2022-05-09,LSN22,810.70,815.30,780.0,807.70,270.0,1690.0
2,2022-05-06,LSN22,853.10,853.10,824.9,827.10,243.0,1681.0
3,2022-05-05,LSN22,904.60,915.00,868.4,873.90,155.0,1670.0
4,2022-05-04,LSN22,923.00,923.00,863.0,909.70,281.0,1644.0
...,...,...,...,...,...,...,...,...
13255,1969-10-07,LSK70,81.75,82.00,81.5,82.00,0.0,0.0
13256,1969-10-06,LSK70,81.50,81.50,81.5,81.50,0.0,0.0
13257,1969-10-03,LSK70,81.50,81.50,81.5,81.50,0.0,0.0
13258,1969-10-02,LSK70,81.00,81.25,81.0,81.25,0.0,0.0


In [36]:
# Takes 40 seconds
backAdjustDays = 1       # 0 means expired on the day of expiry
NearestContractShifted_df = NearestContract_df.copy(deep=True)
shifted_contracts = pd.Series(NearestContractShifted_df.active[0:backAdjustDays+1].values)
NearestContractShifted_df.active = NearestContractShifted_df.active.shift(periods=(backAdjustDays+1))
NearestContractShifted_df.active.iloc[0:backAdjustDays+1] = shifted_contracts
NearestContractShifted_df.active.reset_index(drop=True, inplace=True)

for i in range(len(NearestContractShifted_df)):
    for ohlcv_opt in ['open','high','low','close','volume','openInterest']:
        NearestContractShifted_df.loc[i, ohlcv_opt] = new_combined_pivot[ohlcv_opt].loc[NearestContractShifted_df.iloc[i][0],NearestContractShifted_df.iloc[i][1]]

NearestContractShifted_df

C:\Users\Fanjum_cwp\AppData\Local\Temp\2\ipykernel_2236\1702125489.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NearestContractShifted_df.active.iloc[0:backAdjustDays+1] = shifted_contracts


KeyError: 1008.0

In [33]:
NearestContract_df

,active,open,high,low,close,volume,openInterest
dt,,,,,,,
2022-05-10,LSK22,1008.00,1025.00,1008.0,1022.0,53.0,0.0
2022-05-09,LSK22,1000.10,1015.00,975.0,1009.0,89.0,163.0
2022-05-06,LSK22,990.00,1005.00,985.0,1000.1,147.0,237.0
2022-05-05,LSK22,1010.00,1033.40,993.8,1020.0,131.0,340.0
2022-05-04,LSK22,1021.60,1025.50,980.0,1010.0,144.0,443.0
...,...,...,...,...,...,...,...
1969-10-07,LSH70,78.50,79.00,78.0,79.0,0.0,0.0
1969-10-06,LSH70,80.00,80.50,80.0,80.0,0.0,0.0
1969-10-03,LSH70,80.00,80.00,80.0,80.0,0.0,0.0


In [35]:
# (NearestContractShifted_df == NearestContract_df).all()

In [ ]:
NearestContractShifted_df

In [ ]:
# (NearestContract_df == NearestContractShifted_df).all()

In [ ]:
len(NearestContract_df), len(NearestContractShifted_df)

In [ ]:
NearestContractShifted_df

In [ ]:
expiry_nearest

In [30]:

NearestContract_df.index = NearestContract_df.dt
NearestContract_df.drop('dt', axis=1,inplace = True)

NearestContractShifted_df.index = NearestContractShifted_df.dt
NearestContractShifted_df.drop('dt', axis=1,inplace = True)

# ComparisonNearestContractdf=NearestContract_df.compare(NearestContractShifted_df)
# ComparisonNearestContractdf

In [31]:
# Find Index

idx_to_compare = NearestContract_df.active.compare(NearestContractShifted_df.active).index

compare_columns = []
for t in NearestContract_df.columns:
    compare_columns.append(tuple([t,'self']))
    compare_columns.append(tuple([t,'other']))

ComparisonNearestContractdf = pd.DataFrame(index=idx_to_compare, columns=pd.MultiIndex.from_tuples(compare_columns))

for idx in idx_to_compare:
    for t in NearestContract_df.columns:
        ComparisonNearestContractdf.loc[idx,tuple([t,'self'])] = NearestContract_df.loc[idx,t]
        ComparisonNearestContractdf.loc[idx,tuple([t,'other'])] = NearestContractShifted_df.loc[idx,t]

In [32]:
ComparisonNearestContractdf

active           open            high             low          \
             self  other    self   other    self   other    self   other   
dt                                                                         
2022-03-15  LSH22  LSK22  1420.0  1103.6  1420.0  1160.4  1420.0  1076.6   
2022-03-14  LSH22  LSK22  1348.0  1125.0  1430.0  1129.2  1348.0  1073.2   
2022-01-14  LSF22  LSH22  1270.0  1337.6  1289.0  1338.2  1210.0  1289.0   
2022-01-13  LSF22  LSH22  1244.7  1308.6  1244.7  1329.0  1217.0  1283.9   
2021-11-15  LSX21  LSF22   531.1   621.1   550.0   644.1   531.1   620.0   
...           ...    ...     ...     ...     ...     ...     ...     ...   
1970-07-14  LSN70  LSU70   68.25    75.5    70.0   75.75   68.25    75.2   
1970-05-15  LSK70  LSN70    73.5    78.5    74.0    78.5    72.5    77.0   
1970-05-14  LSK70  LSN70    73.5    77.5   73.75   78.25    73.0    77.5   
1970-03-13  LSH70  LSK70    70.5    77.5    75.5   79.25    70.5    77.5   
1970-03-12  LSH70  LSK70    75.5    79.0    75.5   79.75    75.0    79.0   

             close         volume        openInterest          
              self   other   self  other         self   other  
dt                                                             
2022-03-15  1420.0  1160.4    0.0  213.0         24.0  1556.0  
2022-03-14  1410.0  1103.4   33.0  346.0         24.0  1520.0  
2022-01-14  1237.1  1308.7   34.0  242.0          0.0  1941.0  
2022-01-13  1229.1  1329.0   84.0  337.0         69.0  1895.0  
2021-11-15   536.0   637.0   20.0  120.0         54.0  1271.0  
...            ...     ...    ...    ...          ...     ...  
1970-07-14   68.25    75.5    0.0    0.0          0.0     0.0  
1970-05-15    73.5    78.5    0.0    0.0          0.0     0.0  
1970-05-14    73.5    77.5    0.0    0.0          0.0     0.0  
1970-03-13    70.5    77.5    0.0    0.0          0.0     0.0  
1970-03-12    75.5    79.0    0.0    0.0          0.0     0.0  

[628 rows x 14 columns]

In [ ]:
ComparisonNearestContractdf

In [ ]:
ComparisonNearestContractdf.loc['1989-11-14']


In [ ]:
ComparisonNearestContractdf

In [ ]:
# ComparisonNearestContractdf=NearestContract_df.compare(NearestContractShifted_df)
# ComparisonNearestContractdf

In [ ]:
for ohlcv_opt in ['open','high','low','close','volume','openInterest']:
    ComparisonNearestContractdf[str('Diff_'+ohlcv_opt)] = ComparisonNearestContractdf.loc[:,(ohlcv_opt,'other')].sub(ComparisonNearestContractdf.loc[:,(ohlcv_opt,'self')], axis = 0)
    ComparisonNearestContractdf[str('Prod_'+ohlcv_opt)] = ComparisonNearestContractdf.loc[:,(ohlcv_opt,'other')].div(ComparisonNearestContractdf.loc[:,(ohlcv_opt,'self')], axis = 0)

ComparisonNearestContractdf


# Cumulative Adjustment - Choice Used 'Substraction'

In [ ]:
backAdjustDays = 0
weights = [0]* backAdjustDays + [1]*1 # For 100% weight on the last day
weights

In [ ]:
cumulative_df = {}
final_df = {}
ohlcv_DF = pd.DataFrame()

# weights options
weights = [1/(backAdjustDays+1)]*(backAdjustDays+1) # equally weighted

# Use this as default
weights = [0]* backAdjustDays + [1]*1 # For 100% weight on the last day


#delete this 
weights = [0.6,0.4]

# ,'volume','openInterest' - no adjustment, add unadjusted at the end

for ohlcv_opt in ['close','open','high','low']:

    column_name = str('Diff_'+ohlcv_opt)
    
    # Note that for all days from expiry to backAdjustDays, the multiplier is based on that particular day. From backAdjustDays onwards till next expiry, the multiplier is based on weighted average
    cumulative_df[ohlcv_opt] = pd.DataFrame(index = ComparisonNearestContractdf.index, data = ComparisonNearestContractdf[column_name])
    cumulative_df[ohlcv_opt].reset_index(inplace=True)

    # Put weighted average for all days from expiry till backAdjustDays
    cumulative_df[ohlcv_opt]['wt_avg'] = cumulative_df[ohlcv_opt][column_name].groupby(cumulative_df[ohlcv_opt].index//(backAdjustDays+1)).apply(lambda x: np.multiply(x,weights)).groupby(cumulative_df[ohlcv_opt].index//(backAdjustDays+1)).transform('sum')
    cumulative_df[ohlcv_opt]['Cum_effect'] = cumulative_df[ohlcv_opt]['wt_avg']

    # flag == 1 represents (expiry + backAdjustDays)
    pivot_backAdjustDays_idx = [x for x in cumulative_df[ohlcv_opt].index if (x % (backAdjustDays+1)) == backAdjustDays]

    cumulative_df[ohlcv_opt]['flag'] = [1 if x in pivot_backAdjustDays_idx else 0 for x in cumulative_df[ohlcv_opt].index]

    cumulative_df[ohlcv_opt].loc[cumulative_df[ohlcv_opt]['flag'] == 0, 'Cum_effect'] = 0 # Put 0 for all days before backAdjustDays so as to make cumsum work
    cumulative_df[ohlcv_opt]['Cum_effect'] = cumulative_df[ohlcv_opt]['Cum_effect'].cumsum()

    # Once we have forward filled the cumulative effect, we can take the cumulative effect for days before backAdjustDays
    
    #The following Line adds weighted effect to days before backAdjustDays
    # cumulative_df[ohlcv_opt].loc[cumulative_df[ohlcv_opt]['flag'] == 0, 'Total_effect'] = cumulative_df[ohlcv_opt]['Cum_effect'] + cumulative_df[ohlcv_opt]['wt_avg']
    
    #The following Line adds daily differece effect to days before backAdjustDays
    cumulative_df[ohlcv_opt].loc[cumulative_df[ohlcv_opt]['flag'] == 0, 'Total_effect'] = cumulative_df[ohlcv_opt]['Cum_effect'] + cumulative_df[ohlcv_opt][str('Diff_'+ohlcv_opt)]
    cumulative_df[ohlcv_opt].loc[cumulative_df[ohlcv_opt]['flag'] == 1, 'Total_effect'] = cumulative_df[ohlcv_opt]['Cum_effect']

    cumulative_df[ohlcv_opt].index  = cumulative_df[ohlcv_opt].dt
    cumulative_df[ohlcv_opt].drop('dt', axis=1,inplace = True)
    
    # Using Total effect of only 'close' column for all OHCL since if we don't use Total effect of 'close' column, we will get issues like "adj_low" being higher than "adj_high" for some dates
    final_df[ohlcv_opt] = pd.DataFrame(index = NearestContract_df.index, data = cumulative_df['close']['Total_effect'] )
    final_df[ohlcv_opt].fillna(method='ffill',inplace = True)
    final_df[ohlcv_opt].fillna(0,inplace = True)
    final_df[ohlcv_opt][ohlcv_opt] = NearestContract_df[ohlcv_opt].values
    final_df[ohlcv_opt][str('Adj_'+ohlcv_opt)] = final_df[ohlcv_opt][ohlcv_opt] + final_df['close']['Total_effect']
    ohlcv_DF = pd.concat([ohlcv_DF,final_df[ohlcv_opt]],axis=1)

ohlcv_DF = pd.concat([ohlcv_DF,NearestContract_df[['volume','openInterest']]],axis=1)
ohlcv_DF.drop(['Total_effect'],axis=1,inplace=True)


In [ ]:
ohlcv_DF[ohlcv_DF.Adj_high < ohlcv_DF.Adj_low]

In [ ]:
ohlcv_DF[120:]

In [ ]:
NearestContract_df.loc['1989-11-14']

In [ ]:
NearestContractShifted_df.loc['1989-11-14']

In [ ]:
NearestContractShifted_df.loc['1989-11-14']['close']/NearestContract_df.loc['1989-11-14']['close']

In [ ]:
ComparisonNearestContractdf['close'].loc['1989-11-14']

In [ ]:
ohlcv_DF.loc['2004-07-15']

In [ ]:
cumulative_df.keys()

In [ ]:
final_df['high'].loc['2004-07-13']

In [ ]:
# final_df['high'][4480:4520]

# why is high lower than the low? Check the data

In [ ]:
oh

In [ ]:
# # Takes 4 mins
# writer=pd.ExcelWriter("C:/source/2x4_data/app/data/Barchart/lumber_unadjusted_ohlcv.xlsx") 
# for key in new_combined_pivot.keys():
#     new_combined_pivot[key].to_excel(writer,sheet_name=key)

# writer.save()
# writer.close()

In [ ]:

ohlcv_DF[ohlcv_DF.Adj_high < ohlcv_DF.Adj_low]

In [ ]:
location = "C:/source/2x4_data/app/data/Barchart/ohlcv_lumber_adjusted.csv"
# ohlcv_DF.to_csv(location)

# PRODUCT Method of Panama

# Joining 2 methods

## Absolute - or difference

In [ ]:
cumulative_df[ohlcv_opt]

In [ ]:
ComparisonNearestContractdf['Prod_close']

In [ ]:
# cumulative_df['close']
ComparisonNearestContractdf['close']

In [ ]:
# # Comparison with Absolute method
# backAdjustDays = 1

# cumulative_df = {}
# final_df = {}
# ohlcv_DF = pd.DataFrame()

# # weights options
# weights = [1/(backAdjustDays+1)]*(backAdjustDays+1) # equally weighted

# # Use this as default
# weights = [0]* backAdjustDays + [1]*1 # For 100% weight on the last day

# # ,'volume','openInterest' - no adjustment, add unadjusted at the end


# #delete this
# weights = [0.6,0.4]

# for ohlcv_opt in ['close','open','high','low']:

#     # CHANGE 1
#     column_name = str('Prod_'+ohlcv_opt)
    
#     # Note that for all days from expiry to backAdjustDays, the multiplier is based on that particular day. From backAdjustDays onwards till next expiry, the multiplier is based on weighted average
#     cumulative_df[ohlcv_opt] = pd.DataFrame(index = ComparisonNearestContractdf.index, data = ComparisonNearestContractdf[column_name])
#     cumulative_df[ohlcv_opt].reset_index(inplace=True)

#     # Put weighted average for all days from expiry till backAdjustDays
#     cumulative_df[ohlcv_opt]['wt_avg'] = cumulative_df[ohlcv_opt][column_name].groupby(cumulative_df[ohlcv_opt].index//(backAdjustDays+1)).apply(lambda x: np.multiply(x,weights)).groupby(cumulative_df[ohlcv_opt].index//(backAdjustDays+1)).transform('sum') 
#     cumulative_df[ohlcv_opt]['Cum_effect'] = cumulative_df[ohlcv_opt]['wt_avg']

#     # flag == 1 represents the day on (expiry + backAdjustDays)
#     pivot_backAdjustDays_idx = [x for x in cumulative_df[ohlcv_opt].index if (x % (backAdjustDays+1)) == backAdjustDays]
#     cumulative_df[ohlcv_opt]['flag'] = [1 if x in pivot_backAdjustDays_idx else 0 for x in cumulative_df[ohlcv_opt].index]

#     #CHANGE 2
#     if(backAdjustDays == 0): # Note since if backAdjustDays = 0, all flags will be 1 and thus they need to be made equal to 1
#         pass # If the backAdjustDays equals 0, then its already 1 so no need to put it as 1 
#     else:
#         cumulative_df[ohlcv_opt].loc[cumulative_df[ohlcv_opt]['flag'] == 0, 'Cum_effect'] = 1 # Put 1 for all days before backAdjustDays so as to make cumprod work

#     cumulative_df[ohlcv_opt]['Cum_effect'] = cumulative_df[ohlcv_opt]['Cum_effect'].cumprod()

#     # Once we have forward filled the cumulative effect, we can take the cumulative effect for days before backAdjustDays
    
#     #Change 3
#     #The following Line adds weighted effect to days before backAdjustDays
#     # cumulative_df[ohlcv_opt].loc[cumulative_df[ohlcv_opt]['flag'] == 0, 'Total_effect'] = cumulative_df[ohlcv_opt]['Cum_effect'] * cumulative_df[ohlcv_opt]['wt_avg']
    
#     # Change 4
#     if(backAdjustDays == 0):
#         cumulative_df[ohlcv_opt].loc[cumulative_df[ohlcv_opt]['flag'] == 1, 'Total_effect'] = cumulative_df[ohlcv_opt]['Cum_effect']
#     else:
#         #The following Line adds daily product effect to days before backAdjustDays
#         cumulative_df[ohlcv_opt].loc[cumulative_df[ohlcv_opt]['flag'] == 0, 'Total_effect'] = cumulative_df[ohlcv_opt]['Cum_effect'] * cumulative_df[ohlcv_opt][str('Prod_'+ohlcv_opt)]    
#         cumulative_df[ohlcv_opt].loc[cumulative_df[ohlcv_opt]['flag'] == 1, 'Total_effect'] = cumulative_df[ohlcv_opt]['Cum_effect']
    
    
#     cumulative_df[ohlcv_opt].index  = cumulative_df[ohlcv_opt].dt
#     cumulative_df[ohlcv_opt].drop('dt', axis=1,inplace = True)

#     # Using Total effect of only 'close' column for all OHCL since if we don't use Total effect of 'close' column, we will get issues like "adj_low" being higher than "adj_high" for some dates
#     final_df[ohlcv_opt] = pd.DataFrame(index = NearestContract_df.index, data = cumulative_df['close']['Total_effect'])
#     final_df[ohlcv_opt].fillna(method='ffill',inplace = True)
#     final_df[ohlcv_opt].fillna(0,inplace = True)

    
#     final_df[ohlcv_opt].loc[final_df[ohlcv_opt]['Total_effect'] == 0, 'Total_effect'] = 1    # Add 1 to Total Effect wherever its 0
#     final_df[ohlcv_opt][ohlcv_opt] = NearestContract_df[ohlcv_opt].values

#     # Change 5
#     final_df[ohlcv_opt][str('Adj_'+ohlcv_opt)] = final_df[ohlcv_opt][ohlcv_opt] * final_df['close']['Total_effect']
    
#     ohlcv_DF = pd.concat([ohlcv_DF,final_df[ohlcv_opt]],axis=1)

# ohlcv_DF = pd.concat([ohlcv_DF,NearestContract_df[['volume','openInterest']]],axis=1)
# ohlcv_DF.drop(['Total_effect'],axis=1,inplace=True)


In [ ]:

backAdjustDays = 1

cumulative_df = {}
final_df = {}
ohlcv_DF = pd.DataFrame()

# # Use this as default
weights = [0]* backAdjustDays + [1]*1 # For 100% weight on the last day

for ohlcv_opt in ['close','open','high','low']:

    # CHANGE 1
    column_name = str('Prod_'+ohlcv_opt)
    
    # Note that for all days from expiry to backAdjustDays, the multiplier is based on that particular day. From backAdjustDays onwards till next expiry, the multiplier is based on weighted average
    cumulative_df[ohlcv_opt] = pd.DataFrame(index = ComparisonNearestContractdf.index, data = ComparisonNearestContractdf[column_name])
    cumulative_df[ohlcv_opt].reset_index(inplace=True)

    # Put weighted average for all days from expiry till backAdjustDays
    cumulative_df[ohlcv_opt]['wt_avg'] = cumulative_df[ohlcv_opt][column_name].groupby(cumulative_df[ohlcv_opt].index//(backAdjustDays+1)).apply(lambda x: np.multiply(x,weights)).groupby(cumulative_df[ohlcv_opt].index//(backAdjustDays+1)).transform('sum') 
    cumulative_df[ohlcv_opt]['Cum_effect'] = cumulative_df[ohlcv_opt]['wt_avg']

    # flag == 1 represents the day on (expiry + backAdjustDays)
    pivot_backAdjustDays_idx = [x for x in cumulative_df[ohlcv_opt].index if (x % (backAdjustDays+1)) == backAdjustDays]
    cumulative_df[ohlcv_opt]['flag'] = [1 if x in pivot_backAdjustDays_idx else 0 for x in cumulative_df[ohlcv_opt].index]

    #CHANGE 2
    if(backAdjustDays == 0): # Note since if backAdjustDays = 0, all flags will be 1 and thus they need to be made equal to 1
        pass # If the backAdjustDays equals 0, then its already 1 so no need to put it as 1 
    else:
        cumulative_df[ohlcv_opt].loc[cumulative_df[ohlcv_opt]['flag'] == 0, 'Cum_effect'] = 1 # Put 1 for all days before backAdjustDays so as to make cumprod work

    cumulative_df[ohlcv_opt]['Cum_effect'] = cumulative_df[ohlcv_opt]['Cum_effect'].cumprod()

    # Once we have forward filled the cumulative effect, we can take the cumulative effect for days before backAdjustDays
    
    #Change 3
    #The following Line adds weighted effect to days before backAdjustDays
    # cumulative_df[ohlcv_opt].loc[cumulative_df[ohlcv_opt]['flag'] == 0, 'Total_effect'] = cumulative_df[ohlcv_opt]['Cum_effect'] * cumulative_df[ohlcv_opt]['wt_avg']
    
    # Change 4
    if(backAdjustDays == 0):
        cumulative_df[ohlcv_opt].loc[cumulative_df[ohlcv_opt]['flag'] == 1, 'Total_effect'] = cumulative_df[ohlcv_opt]['Cum_effect']
    else:
        #The following Line adds daily product effect to days before backAdjustDays
        cumulative_df[ohlcv_opt].loc[cumulative_df[ohlcv_opt]['flag'] == 0, 'Total_effect'] = cumulative_df[ohlcv_opt]['Cum_effect'] * cumulative_df[ohlcv_opt][str('Prod_'+ohlcv_opt)]    
        cumulative_df[ohlcv_opt].loc[cumulative_df[ohlcv_opt]['flag'] == 1, 'Total_effect'] = cumulative_df[ohlcv_opt]['Cum_effect']
    
    
    cumulative_df[ohlcv_opt].index  = cumulative_df[ohlcv_opt].dt
    cumulative_df[ohlcv_opt].drop('dt', axis=1,inplace = True)

    # Using Total effect of only 'close' column for all OHCL since if we don't use Total effect of 'close' column, we will get issues like "adj_low" being higher than "adj_high" for some dates
    final_df[ohlcv_opt] = pd.DataFrame(index = NearestContract_df.index, data = cumulative_df['close']['Total_effect'])


    final_df[ohlcv_opt].fillna(method='ffill',inplace = True)
    final_df[ohlcv_opt].iloc[np.argwhere(final_df[ohlcv_opt].index > cumulative_df[ohlcv_opt].index[0])] = 1 # Add 1 to Total Effect prior to first expiry date
    
    
    final_df[ohlcv_opt]['Total_effect'].replace(to_replace=0, method='ffill', inplace=True)
    
    # This may also be a problem
    # final_df[ohlcv_opt].fillna(0,inplace = True)

    # This is the PROBLEM - I think - when you put 1 as Total_effect, it misses the cumprod.
    # final_df[ohlcv_opt].loc[final_df[ohlcv_opt]['Total_effect'] == 0, 'Total_effect'] = 1    # Add 1 to Total Effect wherever its 0

    
    final_df[ohlcv_opt][ohlcv_opt] = NearestContract_df[ohlcv_opt].values

    
    # Change 5
    final_df[ohlcv_opt][str('Adj_'+ohlcv_opt)] = final_df[ohlcv_opt][ohlcv_opt] * final_df['close']['Total_effect']
    
    ohlcv_DF = pd.concat([ohlcv_DF,final_df[ohlcv_opt]],axis=1)

ohlcv_DF = pd.concat([ohlcv_DF,NearestContract_df[['volume','openInterest']]],axis=1)
ohlcv_DF.drop(['Total_effect'],axis=1,inplace=True)


1989-11-14

In [ ]:
ComparisonNearestContractdf

In [ ]:
final_df['close'].loc['']

In [ ]:
final_df['close']['Total_effect'].replace(0,pd.np.nan).ffill(inplace = True)

In [ ]:
final_df['close'].loc['1969-10-20']

In [ ]:
ohlcv_DF.loc['1982-03-12']

In [ ]:
ohlcv_opt ='close'
final_df[ohlcv_opt].iloc[np.argwhere(final_df[ohlcv_opt].index > cumulative_df[ohlcv_opt].index[0])] = 8

In [ ]:
final_df[ohlcv_opt].columns

In [ ]:
# final_df[ohlcv_opt].ix[np.argwhere(final_df[ohlcv_opt].index > cumulative_df[ohlcv_opt].index[0]),np.argwhere(final_df[ohlcv_opt].columns.equals('Total_effect'))] =8

In [ ]:
final_df[ohlcv_opt].iloc[0]

In [ ]:
final_df['close']

In [ ]:
cumulative_df['close']

In [ ]:
final_df['close'].loc['1982-03-12']

In [ ]:
NearestContract_df.loc['1982-03-16']

In [ ]:
ohlcv_DF.loc['1982-03-15']

In [ ]:
final_df[]

In [ ]:
final_df['close'][final_df['close']['Total_effect'] == 0]

In [ ]:
len(NearestContract_df), len(final_df['close'])

In [ ]:
ohlcv_DF

In [ ]:
ohlcv_DF.loc['1989-11-14']

In [ ]:
final_df['close'].loc['1989-11-14']

In [ ]:
ohlcv_DF[8180:]

In [ ]:
ohlcv_DF[78:]

In [ ]:
# faisal

In [ ]:
ohlcv_DF[120:]

In [ ]:
ComparisonNearestContractdf['Prod_close'].loc['1989-11-15']

In [ ]:
cumulative_df['close'][cumulative_df['close']['Prod_close'].isnull()]

In [ ]:
len(cumulative_df['close']), len(ComparisonNearestContractdf['close'])

In [ ]:
cumulative_df['close']

In [ ]:
cumulative_df['close'][385:]

In [ ]:
final_df.keys()

In [ ]:
final_df['close'].loc['1989-11-14']

In [ ]:


cumulative_df = {}
final_df = {}
ohlcv_DF = pd.DataFrame()

# weights options
weights = [1/(backAdjustDays+1)]*(backAdjustDays+1) # equally weighted

# Use this as default
weights = [0]* backAdjustDays + [1]*1 # For 100% weight on the last day

for ohlcv_opt in ['close','open','high','low']:

    # CHANGE 1
    column_name = str('Prod_'+ohlcv_opt)
    
    # Note that for all days from expiry to backAdjustDays, the multiplier is based on that particular day. From backAdjustDays onwards till next expiry, the multiplier is based on weighted average
    cumulative_df[ohlcv_opt] = pd.DataFrame(index = ComparisonNearestContractdf.index, data = ComparisonNearestContractdf[column_name])
    cumulative_df[ohlcv_opt].reset_index(inplace=True)

    # Put weighted average for all days from expiry till backAdjustDays
    cumulative_df[ohlcv_opt]['wt_avg'] = cumulative_df[ohlcv_opt][column_name].groupby(cumulative_df[ohlcv_opt].index//(backAdjustDays+1)).apply(lambda x: np.multiply(x,weights)).groupby(cumulative_df[ohlcv_opt].index//(backAdjustDays+1)).transform('sum') 
    cumulative_df[ohlcv_opt]['Cum_effect'] = cumulative_df[ohlcv_opt]['wt_avg']

    # # flag == 1 represents the day on (expiry + backAdjustDays)
    pivot_backAdjustDays_idx = [x for x in cumulative_df[ohlcv_opt].index if (x % (backAdjustDays+1)) == backAdjustDays]
    cumulative_df[ohlcv_opt]['flag'] = [1 if x in pivot_backAdjustDays_idx else 0 for x in cumulative_df[ohlcv_opt].index]

    # #CHANGE 2
    if(backAdjustDays == 0): # Note since if backAdjustDays = 0, all flags will be 1 and thus they need to be made equal to 1
        pass # If the backAdjustDays equals 0, then its already 1 so no need to put it as 1 
    else:
        cumulative_df[ohlcv_opt].loc[cumulative_df[ohlcv_opt]['flag'] == 0, 'Cum_effect'] = 1 # Put 1 for all days before backAdjustDays so as to make cumprod work

    cumulative_df[ohlcv_opt]['Cum_effect'] = cumulative_df[ohlcv_opt]['Cum_effect'].cumprod()

    #   #Change 3
    # #The following Line adds weighted effect to days before backAdjustDays
    # # cumulative_df[ohlcv_opt].loc[cumulative_df[ohlcv_opt]['flag'] == 0, 'Total_effect'] = cumulative_df[ohlcv_opt]['Cum_effect'] * cumulative_df[ohlcv_opt]['wt_avg']
    
    # Change 4
    if(backAdjustDays == 0):
        cumulative_df[ohlcv_opt].loc[cumulative_df[ohlcv_opt]['flag'] == 1, 'Total_effect'] = cumulative_df[ohlcv_opt]['Cum_effect']
        pass
    else:
        #The following Line adds daily product effect to days before backAdjustDays
        cumulative_df[ohlcv_opt].loc[cumulative_df[ohlcv_opt]['flag'] == 0, 'Total_effect'] = cumulative_df[ohlcv_opt]['Cum_effect'] * cumulative_df[ohlcv_opt][str('Prod_'+ohlcv_opt)]    
        cumulative_df[ohlcv_opt].loc[cumulative_df[ohlcv_opt]['flag'] == 1, 'Total_effect'] = cumulative_df[ohlcv_opt]['Cum_effect']
    
    
    cumulative_df[ohlcv_opt].index  = cumulative_df[ohlcv_opt].dt
    cumulative_df[ohlcv_opt].drop('dt', axis=1,inplace = True)

    # Using Total effect of only 'close' column for all OHCL since if we don't use Total effect of 'close' column, we will get issues like "adj_low" being higher than "adj_high" for some dates
    final_df[ohlcv_opt] = pd.DataFrame(index = NearestContract_df.index, data = cumulative_df['close']['Total_effect'])
    final_df[ohlcv_opt].fillna(method='ffill',inplace = True)
    
    # NEW
    final_df[ohlcv_opt].iloc[np.argwhere(final_df[ohlcv_opt].index > cumulative_df[ohlcv_opt].index[0])] = 1 # Add 1 to Total Effect prior to first expiry date

    # Not sure what this does?
    # final_df[ohlcv_opt].fillna(0,inplace = True)
    
    # final_df[ohlcv_opt].loc[final_df[ohlcv_opt]['Total_effect'] == 0, 'Total_effect'] = 1    # Add 1 to Total Effect wherever its 0
    

    
    final_df[ohlcv_opt][ohlcv_opt] = NearestContract_df[ohlcv_opt].values


In [ ]:
cumulative_df[ohlcv_opt].index[0]

In [ ]:


final_df[ohlcv_opt].iloc[np.where(final_df[ohlcv_opt]['Total_effect'].isna())] = 1

In [ ]:
final_df['close']

In [ ]:
np.where(final_df[ohlcv_opt].index)

(cumulative_df[ohlcv_opt].index[0])

In [ ]:
ohlcv_opt = 'close'
final_df[ohlcv_opt].loc[:cumulative_df[ohlcv_opt].index[0]][:-1].loc['Total_effect'] = 1
final_df[ohlcv_opt][:cumulative_df[ohlcv_opt].index[0]][:-1]['Total_effect']


In [ ]:
final_df[ohlcv_opt][final_df[ohlcv_opt]['Total_effect'].isnull()]

In [ ]:
ohlcv_DF

In [ ]:
NearestContract_df

In [ ]:
# cumulative_df['close']

In [ ]:
# pivot_backAdjustDays_idx

In [ ]:
ohlcv_DF[38:]

In [ ]:
final_df['close']

In [ ]:
cumulative_df['close']

In [ ]:
ohlcv_DF

In [ ]:
ohlcv_DF[38:]

In [ ]:


# Note that for all days from expiry to backAdjustDays, the multiplier remains the same. It also remains same from backadjustDays till next expiry

column_name = 'Prod'
cumulative_df = pd.DataFrame(index = ComparisonNearestContractdf.index, data = ComparisonNearestContractdf[column_name])
cumulative_df.reset_index(inplace=True)

# Put weighted average for all days from expiry till backAdjustDays
cumulative_df['wt_avg'] = cumulative_df[column_name].groupby(cumulative_df.index//(backAdjustDays+1)).apply(lambda x: np.multiply(x,weights)).groupby(cumulative_df.index//(backAdjustDays+1)).transform('sum')
cumulative_df['Cum_effect'] = cumulative_df['wt_avg']

# flag == 1 represents (expiry + backAdjustDays)
pivot_backAdjustDays_idx = [x for x in cumulative_df.index if (x % (backAdjustDays+1)) == backAdjustDays]
cumulative_df['flag'] = [1 if x in pivot_backAdjustDays_idx else 0 for x in cumulative_df.index]

cumulative_df.loc[cumulative_df['flag'] == 0, 'Cum_effect'] = 1 # Put 1 for all days before backAdjustDays so as to make cumprod work
cumulative_df['Cum_effect'] = cumulative_df['Cum_effect'].cumprod()

# Once we have forward filled the cumulative effect, we can take the cumulative effect for days before backAdjustDays
cumulative_df.loc[cumulative_df['flag'] == 0, 'Total_effect'] = cumulative_df['Cum_effect'] * cumulative_df['wt_avg']
cumulative_df.loc[cumulative_df['flag'] == 1, 'Total_effect'] = cumulative_df['Cum_effect']

cumulative_df.index  = cumulative_df.dt
cumulative_df.drop('dt', axis=1,inplace = True)

final_df = pd.DataFrame(index = NearestContract_df.index, data = cumulative_df['Total_effect'] )
final_df.fillna(method='ffill',inplace = True)
final_df.fillna(1,inplace = True)
final_df['close'] = NearestContract_df['close'].values
final_df['Adjusted'] = final_df['close'] * final_df['Total_effect']

final_df


In [ ]:
ohlcv_DF

In [ ]:


# Note that for all days from expiry to backAdjustDays, the multiplier remains the same. It also remains same from backadjustDays till next expiry

column_name = 'Prod'
cumulative_df = pd.DataFrame(index = ComparisonNearestContractdf.index, data = ComparisonNearestContractdf[column_name])
cumulative_df.reset_index(inplace=True)

# Put weighted average for all days from expiry till backAdjustDays
cumulative_df['wt_avg'] = cumulative_df[column_name].groupby(cumulative_df.index//(backAdjustDays+1)).apply(lambda x: np.multiply(x,weights)).groupby(cumulative_df.index//(backAdjustDays+1)).transform('sum')
cumulative_df['Cum_effect'] = cumulative_df['wt_avg']

# flag == 1 represents (expiry + backAdjustDays)
pivot_backAdjustDays_idx = [x for x in cumulative_df.index if (x % (backAdjustDays+1)) == backAdjustDays]
cumulative_df['flag'] = [1 if x in pivot_backAdjustDays_idx else 0 for x in cumulative_df.index]

cumulative_df.loc[cumulative_df['flag'] == 0, 'Cum_effect'] = 1 # Put 1 for all days before backAdjustDays so as to make cumprod work
cumulative_df['Cum_effect'] = cumulative_df['Cum_effect'].cumprod()

# Once we have forward filled the cumulative effect, we can take the cumulative effect for days before backAdjustDays
cumulative_df.loc[cumulative_df['flag'] == 0, 'Total_effect'] = cumulative_df['Cum_effect'] * cumulative_df['wt_avg']
cumulative_df.loc[cumulative_df['flag'] == 1, 'Total_effect'] = cumulative_df['Cum_effect']

cumulative_df.index  = cumulative_df.dt
cumulative_df.drop('dt', axis=1,inplace = True)

final_df = pd.DataFrame(index = NearestContract_df.index, data = cumulative_df['Total_effect'] )
final_df.fillna(method='ffill',inplace = True)
final_df.fillna(1,inplace = True)
final_df['close'] = NearestContract_df['close'].values
final_df['Adjusted'] = final_df['close'] * final_df['Total_effect']

final_df


## Relative or product method

In [ ]:


# Note that for all days from expiry to backAdjustDays, the multiplier remains the same. It also remains same from backadjustDays till next expiry

column_name = 'Prod'
cumulative_df = pd.DataFrame(index = ComparisonNearestContractdf.index, data = ComparisonNearestContractdf[column_name])
cumulative_df.reset_index(inplace=True)

# Put weighted average for all days from expiry till backAdjustDays
cumulative_df['wt_avg'] = cumulative_df[column_name].groupby(cumulative_df.index//(backAdjustDays+1)).apply(lambda x: np.multiply(x,weights)).groupby(cumulative_df.index//(backAdjustDays+1)).transform('sum')
cumulative_df['Cum_effect'] = cumulative_df['wt_avg']

# flag == 1 represents (expiry + backAdjustDays)
pivot_backAdjustDays_idx = [x for x in cumulative_df.index if (x % (backAdjustDays+1)) == backAdjustDays]
cumulative_df['flag'] = [1 if x in pivot_backAdjustDays_idx else 0 for x in cumulative_df.index]

cumulative_df.loc[cumulative_df['flag'] == 0, 'Cum_effect'] = 1 # Put 1 for all days before backAdjustDays so as to make cumprod work
cumulative_df['Cum_effect'] = cumulative_df['Cum_effect'].cumprod()

# Once we have forward filled the cumulative effect, we can take the cumulative effect for days before backAdjustDays
cumulative_df.loc[cumulative_df['flag'] == 0, 'Total_effect'] = cumulative_df['Cum_effect'] * cumulative_df['wt_avg']
cumulative_df.loc[cumulative_df['flag'] == 1, 'Total_effect'] = cumulative_df['Cum_effect']

cumulative_df.index  = cumulative_df.dt
cumulative_df.drop('dt', axis=1,inplace = True)

final_df = pd.DataFrame(index = NearestContract_df.index, data = cumulative_df['Total_effect'] )
final_df.fillna(method='ffill',inplace = True)
final_df.fillna(1,inplace = True)
final_df['close'] = NearestContract_df['close'].values
final_df['Adjusted'] = final_df['close'] * final_df['Total_effect']

final_df


In [ ]:
# pd.MultiIndex.from_frame(ComparisonNearestContractdf)

ComparisonNearestContractdf.columns.get_level_values(0), ComparisonNearestContractdf.columns.get_level_values(1)

In [ ]:
ComparisonNearestContractdf.iloc[:, ComparisonNearestContractdf.columns.get_level_values(1)=='self']

In [ ]:
# Check if any of the expiries are less than 2 std dev away from the mean of the date difference between 2 consecutive expiry months

def zscore(s):
    return (s - np.mean(s)) / np.std(s)

# Check if difference in dates is more or less similar - left starting 2 dates - becasue one will be NaN and other is current day so wil be less
expiry_month_diff = pd.to_datetime(new_combined_pivot.loc['Expiry']).diff().dt.days[2:].values
pd.Series(expiry_month_diff)[zscore(pd.Series(expiry_month_diff)).abs().gt(2)]

In [ ]:
columns = ['Cmdty','M1','M2','M3','M4','M5','M6','M7','M8','M9','M10','M11','M12']
one_cmdty_df = pd.DataFrame(columns=columns, index=all_dates)
cmdty_calender_mnths = pd.DataFrame(columns=columns)

In [ ]:
cmdty_calender_mnths

In [ ]:
# cmdty_ctrct_dict['AUD_USD Micro']
cmdty_ctrct_expiry_dict['AUD_USD Micro']['MGM21']

In [ ]:
cmdty_date_df = pd.DataFrame(columns=['Dates'])
cmdty_date_df.loc['2','Dates'] = 1
# cmdty_date_df

In [ ]:
import datetime
def date_less_than(date, ctrct_expiry):
    return date <= ctrct_expiry 

# def date_greater_than(end, current):
#     return current <= end

stick together continue & nearest contracts